In [ ]:
!pip install transformers torch indic-nlp-library sentencepiece gradio datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading the data**

In [ ]:
import pandas as pd

# Load the dataset
try:
    df = pd.read_csv('/content/hindi_news_dataset.csv', encoding='utf-8',nrows=129934) #add if in dataset error ,nrows=129934
    print("Dataset (first 5 rows):")
    print(df.head())
    print("\nColumns:", df.columns.tolist())
    print("Dataset Size:", len(df))
except Exception as e:
    print("Error loading CSV:", e)


Dataset (first 5 rows):
                                            Headline  \
0  कांग्रेस नेता बलजिंदर सिंह की पंजाब में घर के ...   
1  केंद्रीय मंत्री बोले- महिला आरक्षण लाने का साह...   
2  ओपीएस लागू करने से अस्थिर हो सकती है राज्यों क...   
3  तमिलनाडु में शावरमा खाने से 14 वर्षीय छात्रा क...   
4  मणिपुर में मुख्यमंत्री के आश्वासन के बाद मारे ...   

                                             Content  \
0  कांग्रेस नेता बलजिंदर सिंह की सोमवार को पंजाब ...   
1  केंद्रीय मंत्री प्रह्लाद पटेल ने लोकसभा और विध...   
2  आरबीआई के 5 अधिकारियों ने एक लेख में लिखा है क...   
3  नामक्कल (तमिलनाडु) में शावरमा खाने से सोमवार क...   
4  मणिपुर के मुख्यमंत्री एन बीरेन सिंह के आश्वासन...   

            News Categories        Date  
0              ['national']  19-09-2023  
1  ['politics', 'national']  19-09-2023  
2  ['business', 'national']  19-09-2023  
3              ['national']  19-09-2023  
4              ['national']  19-09-2023  

Columns: ['Headline', 'Content', 'News Categories

**Hugging Face Authentication**

In [ ]:
from huggingface_hub import login

# Replace with your Hugging Face token
hf_token = "hf_ipSKdsanmmMCHwHHGYYVCZoyZWxzy*****"  # Paste your token here
login(token=hf_token)

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Authenticate with Hugging Face
try:
    login(token=userdata.get('HF_TOKEN'))
    print("Authentication successful!")
except Exception as e:
    print("Authentication failed:", e)

Authentication successful!


# **Preprocessing the data**

In [ ]:
import pandas as pd
from indicnlp.tokenize import indic_tokenize
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# Load dataset
df = pd.read_csv('/content/hindi_news_dataset.csv', encoding='utf-8',nrows=129934) #add if in dataset error ,nrows=129934

# Rename columns
df = df.rename(columns={'Content': 'article', 'Headline': 'summary'})

# Clean text function
def clean_text(text):
    if isinstance(text, str):
        text = text.strip().replace('\n', ' ').replace('\r', '').replace('  ', ' ')
        return text
    return ""

# Apply cleaning
df['article'] = df['article'].apply(clean_text)
df['summary'] = df['summary'].apply(clean_text)

# Remove empty rows
df = df[df['article'].str.len() > 0]
df = df[df['summary'].str.len() > 0]

# Verify cleaned data
print("Cleaned Dataset Size:", len(df))
print("Sample Article:", df['article'].iloc[0])
print("Sample Summary:", df['summary'].iloc[0])

# Use a subset to manage memory
df = df.sample(5000, random_state=42)

# Split into train and test (80% train, 20% test)
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['article', 'summary']])
test_dataset = Dataset.from_pandas(test_df[['article', 'summary']])
dataset = DatasetDict({'train': train_dataset, 'test': test_dataset})

# Load tokenizer
model_name = "ai4bharat/IndicBARTSS"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the source and target language for mBART
tokenizer.src_lang = "hi_IN"  # Hindi
tokenizer.tgt_lang = "hi_IN"  # Hindi

# Tokenize dataset
def preprocess_function(examples):
    inputs = examples['article']
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Cleaned Dataset Size: 129934
Sample Article: कांग्रेस नेता बलजिंदर सिंह की सोमवार को पंजाब के मोगा में उनके घर में गोली मारकर हत्या कर दी गई। ऑनलाइन सामने आए सीसीटीवी फुटेज में बलजिंदर को गोलियां मारता हुआ एक हमलावर दिख रहा है। पुलिस ने बताया, "बलजिंदर को एक गोली सीने में लगी और आशंका है कि दूसरा हमलावर घर के बाहर बाइक पर सवार था।"
Sample Summary: कांग्रेस नेता बलजिंदर सिंह की पंजाब में घर के अंदर गोली मारकर की गई हत्या


tokenizer_config.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# **Fine-Tune and train IndicBARTSS**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from google.colab import userdata

# Load model
model_name = "ai4bharat/IndicBARTSS"
hf_token = userdata.get('HF_TOKEN')  # Get token from Colab Secrets
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, token=hf_token if hf_token else None)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/indicbartss_finetuned",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    gradient_accumulation_steps=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# Start training
trainer.train()

# Save model to Google Drive
model.save_pretrained("/content/drive/My Drive/indicbartss_finetuned")
tokenizer.save_pretrained("/content/drive/My Drive/indicbartss_finetuned")

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/976M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pavova8202 (pavova8202-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,7.302900,4.266286
2,2.819500,1.063501
3,1.041400,0.637006


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


('/content/drive/My Drive/indicbartss_finetuned/tokenizer_config.json',
 '/content/drive/My Drive/indicbartss_finetuned/special_tokens_map.json',
 '/content/drive/My Drive/indicbartss_finetuned/spiece.model',
 '/content/drive/My Drive/indicbartss_finetuned/added_tokens.json',
 '/content/drive/My Drive/indicbartss_finetuned/tokenizer.json')

# **Test the Model**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/My Drive/indicbartss_finetuned")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/indicbartss_finetuned")

# Define summarization function
def summarize_text(article):
    inputs = tokenizer(
        article,
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=150,
        min_length=50,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test on samples
for i in range(2):
    article = test_df['article'].iloc[i]
    reference = test_df['summary'].iloc[i]
    print(f"\nSample {i+1}:")
    print("Article:", article)
    print("Reference Summary:", reference)
    print("Generated Summary:", summarize_text(article))


Sample 1:
Article: पाकिस्तान के पूर्व प्रधानमंत्री इमरान खान की पार्टी 'पाकिस्तान तहरीक-ए-इंसाफ' (पीटीआई) ने पाक सेना पर इमरान के भतीजे हसन नियाज़ी को आर्मी कस्टडी से अगवा करने का आरोप लगाया है। यह आरोप तब सामने आया है जब इमरान ने सेना प्रमुख असीम मुनीर को उनकी पत्नी बुशरा बीबी को जेल में रखने के लिए सीधे तौर पर ज़िम्मेदार ठहराया था।
Reference Summary: पाकिस्तान के पूर्व पीएम इमरान की पार्टी ने सेना पर लगाया उनके भतीजे का अपहरण करने का आरोप
Generated Summary: पाकिस्तान के पूर्व प्रधानमंत्री इमरान खान की पार्टी 'पाकिस्तान तहरीक-ए-इंसाफ' ने पाक सेना पर इमरान के भतीजे की पत्नी बुशरा बीबी को आर्मी कस्टडी से अगवा करने का आरोप लगाया है: इमरान ने पाक सेना पर इमरान के भतीजे हसन नियाज़ी को आर्मी कस्टडी से अगवा करने का आरोप लगाया: इमरान ने जताया त

Sample 2:
Article: भारत ने मंगलवार को कनाडा के शीर्ष राजनयिक को निष्कासित कर दिया। कनाडाई प्रधानमंत्री जस्टिन ट्रूडो ने कहा था कि खालिस्तानी आतंकी हरदीप सिंह निज्जर की हत्या में भारत सरकार के एजेंटों का हाथ हो सकता है जिसके बाद कनाडा ने शीर्ष भारतीय र

# **Creating gradio Interface**

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/My Drive/indicbartss_finetuned")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/indicbartss_finetuned")

# Define summarization function
def summarize_text(article):
    inputs = tokenizer(
        article,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding="max_length"
    )
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=150,
        min_length=50,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Create Gradio interface
interface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(lines=10, placeholder="Paste news article here..."),
    outputs="text",
    title="Indian News Summarization",
    description="Enter a news article to get a concise summary."
)

# Launch interface
interface.launch()